In [21]:
from gene_ii_asym import load_sp_mat

iui = load_sp_mat('datasets/Youshu/iui_asym.npz')
print(iui.getnnz())
ibi = load_sp_mat('datasets/Youshu/ibi_asym.npz')
print(ibi.getnnz())


ii_b_max = int(ibi.max())
print(ii_b_max)
ii_u_max = int(iui.max())
print(ii_u_max)


# bundle
print("statistic")
print("ii bundle")
for i in range(1, 10):
    count = ibi.multiply(ibi == i).getnnz()
    p = count / ibi.getnnz() * 100
    print("==", i, ":", p, "%")

count = ibi.multiply(ibi >= 10).getnnz()
p = count / ibi.getnnz() * 100
print(">=", 10, ":", p, "%")


# user
print("ii user")
for i in range(1, 10):
    count = iui.multiply(iui == i).getnnz()
    p = count / iui.getnnz() * 100
    print("==", i, ":", p, "%")

count = iui.multiply(ibi >= 10).getnnz()
p = count / ibi.getnnz() * 100
print(">=", 10, ":", p, "%")

3154510
8674226
394
778
statistic
ii bundle
== 1 : 83.57963004422527 %
== 2 : 9.677900944706767 %
== 3 : 3.0545434255459796 %
== 4 : 1.375788456514737 %
== 5 : 0.7322958843820763 %
== 6 : 0.43762982426328295 %
== 7 : 0.2806244614793297 %
== 8 : 0.18844332624028934 %
== 9 : 0.13269195430232045 %
>= 10 : 0.5404516783399465 %
ii user
== 1 : 86.58507977467181 %
== 2 : 8.210498619436933 %
== 3 : 2.4250042003353927 %
== 4 : 1.0465333760235345 %
== 5 : 0.5575509350105088 %
== 6 : 0.31805256600866694 %
== 7 : 0.20776602388326554 %
== 8 : 0.13720038928391415 %
== 9 : 0.09906451398156925 %
>= 10 : 0.5104432372409942 %


In [22]:
print(ibi.shape, iui.shape)

(32770, 32770) (32770, 32770)


In [23]:
from sklearn.preprocessing import normalize
import scipy.sparse as sp
import numpy as np

n_items = ibi.shape[0]
ibi_filter = ibi >= 2
iui_filter = iui >= 2

# mask all diag weight
diag_filter_i = sp.coo_matrix(
    (np.ones(n_items), ([i for i in range(0, n_items)], [i for i in range(0, n_items)])), 
    shape=ibi.shape).tocsr()

fil_iui = iui.multiply(iui_filter)
fil_ibi = ibi.multiply(ibi_filter)

# mask all diag of filtered matrix
diag_filter_iui = fil_iui.multiply(diag_filter_i)
diag_filter_ibi = fil_ibi.multiply(diag_filter_i)

diag_filter_iui, diag_filter_ibi

(<32770x32770 sparse matrix of type '<class 'numpy.float64'>'
 	with 11419 stored elements in Compressed Sparse Column format>,
 <32770x32770 sparse matrix of type '<class 'numpy.float64'>'
 	with 15176 stored elements in Compressed Sparse Column format>)

In [24]:
print(diag_filter_ibi.getnnz())
print(diag_filter_iui.getnnz())


15176
11419


In [25]:
print(fil_ibi.getnnz())
print(fil_iui.getnnz())

1424340
423175


In [26]:
neighbor_ibi = fil_ibi - diag_filter_ibi.tocsc()
neighbor_iui = fil_iui - diag_filter_iui.tocsc() 
neighbor_ibi.getnnz(), neighbor_iui.getnnz() # -> match

(1409164, 411756)

In [27]:
# norm or not
# n_ibi = normalize(neighbor_ibi, norm='l1', axis=1)
# n_iui = normalize(neighbor_iui, norm='l1', axis=1)

n_ibi = neighbor_ibi
n_iui = neighbor_iui

In [28]:
print(n_ibi[1,:]), print(n_iui[1,:])

  (0, 5793)	2.0
  (0, 2180)	2.0
  (0, 2616)	2.0
  (0, 3463)	3.0
  (0, 5793)	3.0
  (0, 6175)	2.0
  (0, 6740)	2.0
  (0, 7913)	2.0
  (0, 8170)	4.0
  (0, 10240)	2.0
  (0, 11154)	2.0
  (0, 14087)	2.0
  (0, 14501)	2.0
  (0, 16253)	3.0
  (0, 17191)	2.0
  (0, 17495)	2.0
  (0, 18100)	2.0
  (0, 18330)	2.0
  (0, 18761)	2.0
  (0, 19280)	2.0
  (0, 20694)	2.0
  (0, 23872)	2.0
  (0, 23980)	2.0
  (0, 24175)	2.0
  (0, 24336)	2.0
  (0, 24508)	2.0
  (0, 25080)	2.0
  (0, 25752)	3.0
  (0, 26643)	3.0
  (0, 26683)	2.0
  (0, 29737)	2.0
  (0, 30487)	2.0


(None, None)

In [29]:
sp.save_npz("datasets/Youshu/n_neigh_ibi.npz", n_ibi)
sp.save_npz("datasets/Youshu/n_neigh_iui.npz", n_iui)

In [30]:
# # test load
# from gene_ii_asym import load_sp_mat

# n_ibi = load_sp_mat("datasets/Youshu/n_neigh_ibi.npz")
# print(n_ibi)